In [1]:
import numpy as np
import cv2

In [2]:
import insightface

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.13). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.


#### 1. Configure Insight Face Model

In [3]:
from insightface.app import FaceAnalysis

In [4]:
# buffalo_l model
app_l = FaceAnalysis(name='buffalo_l', root='insightface_model', providers=['CPUExecutionProvider'])
app_l.prepare(ctx_id=0, det_size=(640, 640))

download_path: insightface_model/models/buffalo_l


  2%|▏         | 4375/281857 [09:50<10:24:33,  7.40KB/s]


KeyboardInterrupt: 

#### 2. Load Image

In [5]:
img = cv2.imread('test_image_2.jpg')

cv2.imshow('image', img)
cv2.waitKey(0) # as soon as if i press esc key or x button, window will get close
cv2.destroyAllWindows()

error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
